In [ ]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('../')

# Parameters

In [ ]:
np.random.seed(1992)
frl_key='tk5'
period = "1819"
recalculate = False
output_path = "/share/data/school_choice_equity/simulator_data/student/"
student_data_file = "/share/data/school_choice/Data/Cleaned/drop_optout_{}.csv"

# Load Student Data

In [ ]:
from src.d01_data.student_data_api import StudentDataApi

student_df = pd.read_csv(student_data_file.format(period))
mask = student_df['grade'] == 'KG'
student_df = student_df.loc[mask]
student_df.head()

# Generate Individual Focal Students

The individual focal students for each block group are going to be generated at random. We are going to use the block groupes defined by SFUSD. The available data allows us to get rough estimates of the following probabilities for each block group:

- Probability a student is AALPI: $P(AALPI)$
- Probability a student is FRL: $P(FRL)$
- Probability a student is both AALPI and FRL: $P(AALPI \cap FRL)$
- Probability a student is FRL conditional on him being AALPI: $$P(FRL \mid AALPI) = \frac{P(AALPI \cap FRL)}{P(AALPI)}$$
- Probability a student is FRL conditional on him not being AALPI: $$P(FRL \mid \overline{AALPI}) = \frac{P(FRL)-P(AALPI \cap FRL)}{1-P(AALPI)}$$

Since some blocks have a small number of students we are going to use the mean of the posterior probability of a Bernoulli distribution with a uniform prior as probability estimates. Let $\theta$ be the parameter of the Bernoulli disttribution, $m$ the counts of interest and $n$ the total counts, we can get our probability estimates as: 

$$\hat{\theta} = \mathbb{E}\left[\theta\mid m,n\right]=\frac{m+1}{n+2}$$

For example, we can estimate the probability of a student being AALPI as

$$P(AALPI)=\frac{\textit{counts of AALPI}+1}{\textit{total counts}+2}$$

In [ ]:
from src.d00_utils.utils import add_bayesian_bernoulli
from src.d02_intermediate.classifier_data_api import ClassifierDataApi
from src.d00_utils.utils import get_label

def frl_cond_aalpi(row):
    return row['probBoth'] / row['probAALPI']
def frl_cond_naalpi(row):
    return (row['probFRL'] - row['probBoth']) / (1. - row['probAALPI'])
    
def isFocal(row, frl_df):
    geoid = row['census_block']
    if np.isnan(geoid):
        return 0
    geoid = int(geoid)
    u = np.random.random()
    if geoid in frl_df.index:
        probs = frl_df.loc[geoid]
        if row['AALPI'] == 1:
            if u <= probs['probCondAALPI']:
                return 1
            else:
                return 0
        else:
            if u <= probs['probCondNAALPI']:
                return 1
            else:
                return 0
    else:
        return 0

In [ ]:
def add_focal_students(student_df):
    
    cda = ClassifierDataApi()
    frl_df = cda.get_frl_data(frl_key=frl_key)
    frl_df = add_bayesian_bernoulli(frl_df)

    frl_df['probCondAALPI'] = frl_df.apply(lambda row: frl_cond_aalpi(row), axis=1)
    frl_df['probCondNAALPI'] = frl_df.apply(lambda row: frl_cond_naalpi(row), axis=1)

    aalpi_ethnicity_list =  ['Black or African American','Hispanic/Latino','Pacific Islander']

    student_df['AALPI'] = student_df['resolved_ethnicity'].isin(aalpi_ethnicity_list).astype('int64')
    student_df['focal'] = student_df.apply(lambda row: isFocal(row, frl_df), axis=1)

# Add Equity Tiebreaker Column

In [ ]:
def add_equity_tiebreaker(student_df, model, params, tiebreaker):
    solution = model.get_solution_set(params)
    student_df[tiebreaker] = student_df['census_block'].apply(lambda x: get_label(x, solution))

# Update Student Data for Simulations

In [ ]:
def get_file_name(period):
    return "drop_optout_{}.csv".format(period)

def check_consistency(student_out, student_df):
    test = student_out['focal'] != student_df['focal']
    if test.any():
        raise Exception("Previous aved student data is not consisten with student data being generated")
        
def update_student_data(student_df, output_path, period, tiebreaker):
    global recalculate
    fname = get_file_name(period)
    if os.path.isfile(output_path + fname):
        print("Loading student data from:\n %s" % (output_path + fname))
        student_out = pd.read_csv(output_path + fname).set_index('studentno')
        check_consistency(student_out, student_df)
        if tiebreaker not in student_out.columns or recalculate:
            print("Updateing %s in student data..." % tiebreaker)
            student_out[tiebreaker] = student_df[tiebreaker]
            recalculate = False
        else:
            raise Exception("Tiebreaker already exists!")
            
    else:
        print("Creating student data:\n %s" % (output_path + fname))
        student_out = student_df.copy()
        
    return student_out
        
def save_student_data(student_out, output_path, period):
    fname = get_file_name(period)
    print("Saving to:\n  %s" % (output_path + fname))
    student_out.reset_index().to_csv(output_path + fname)

## Compute new columns

In [ ]:
from src.d04_modeling.knapsack_classifier import KnapsackClassifier

add_focal_students(student_df)

tiebreaker = 'knapsack008'
fpr = 0.08
model = KnapsackClassifier(positive_group='nFocal', load=True,
                           frl_key=frl_key, run_name=frl_key+".pkl")

add_equity_tiebreaker(student_df, model, params=fpr, tiebreaker=tiebreaker)
student_df[tiebreaker].mean()

In [ ]:
student_out = update_student_data(student_df, output_path, period, tiebreaker)

In [ ]:
save_student_data(student_out, output_path, period)